In [ ]:
import mlflow
# Connect to Databricks workspace with submitted credentials or use stored credentials
mlflow.login()
# Set tracking URI to Databricks -  tell MLflow to send the data into Databricks Workspace
mlflow.set_tracking_uri("databricks")

In [ ]:
import os
import logging
import mlflow
from datetime import datetime
from dotenv import load_dotenv

logging.basicConfig(level=logging.INFO)
logging.getLogger("mlflow.utils.environment").setLevel(logging.DEBUG)


load_dotenv()

experiment_name = f"/Users/{os.environ['USER_EMAIL']}/agent-no-memory-{datetime.now()}"
try:
    mlflow.create_experiment(experiment_name)
except Exception as e:
    print(f"Experiment {experiment_name} already exists.")
mlflow.set_experiment(experiment_name)


with mlflow.start_run():
    logged_agent_info = mlflow.pyfunc.log_model(
        python_model="agent.py",
        name="Helpful agent without memory",
        input_example={"input": [{"role": "user", "content": "What is the full name of poet Byron?"}]}, # Example input instead of default sample "Hello!"
        registered_model_name="workspace.default.helpful-agent-no-memory",
        code_paths=["agent.py"]
    )